# Different Usecases for Jaccard Distance

## A simple implementation of autocorrect

The [original dataset](https://www.kaggle.com/datasets/anthonytherrien/dictionary-of-english-words-and-definitions) was cleaned up as we only need the words in this case.

Note: in later implementations, we can add more details for the autocorrect suggestions, such as letter ordering, word length, 'letter distance', and even sentence context. Right now we only filter out by word length.

Note that this is nowhere near fast enough to actually use in phones, etc. it's just a naive demonstration of jaccard distance.

In [11]:
import sys 

sys.path.append('..')

def edit_distance(s1, s2):
    if len(s1) < len(s2):
        return edit_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = list(range(len(s2) + 1))

    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

def ngram_jaccard_distance(s1, s2, n=2):
    def get_ngrams(text, n):
        return set([text[i:i+n] for i in range(len(text)-n+1)])
    
    set1 = get_ngrams(s1, n)
    set2 = get_ngrams(s2, n)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return 1 - intersection / union if union > 0 else 0

def suggestion(word):
    with open('../data/dictionary.txt', 'r', encoding='utf-16') as f:
        input_word = word
        dictionary = f.read().splitlines()
        candidates = []
        for candidate in dictionary:
            if abs(len(candidate) - len(input_word)) <= 1:
                distance = edit_distance(input_word, candidate)
                candidates.append((candidate, distance))

        # Sort by edit distance
        candidates.sort(key=lambda x: x[1])

        if candidates and candidates[0][1] == 0 and candidates[0][0] == input_word:
            print(f"The word {input_word} is already in dictionary, but the next top 5 suggestions are:")
            top_suggestions = candidates[1:6]

        else:
            print("Top 5 suggestions:")
            top_suggestions = candidates[:5]

        for candidate, _ in top_suggestions:
            print(candidate)

    # If this was a real function we would return words, of course. 

suggestion('helo')


Top 5 suggestions:
halo
hell
hello
helm
help


## Identifying if two people are family

In [ ]:
import sys
from Bio import Entrez, SeqIO
from io import StringIO

sys.path.append('..')
from distances.jac import jaccard_distance

Entrez.email = "your_real_email@domain.com"  # must be valid per NCBI policy

def fetch_sequences(query, max_samples=5):
    """
    Fetch multiple sequences at once using Entrez, more efficient than one-by-one efetch.
    """
    print(f"[INFO] Searching NCBI for query: '{query}' (max {max_samples})...")
    search_handle = Entrez.esearch(db="nucleotide", term=query, retmax=max_samples)
    record = Entrez.read(search_handle)
    ids = record['IdList']
    print(f"[INFO] Found {len(ids)} sequence IDs.")

    if not ids:
        print("[WARN] No sequences found for the query.")
        return []
    
    print(f"[INFO] Fetching {len(ids)} sequences from NCBI in bulk...")
    fetch_handle = Entrez.efetch(db="nucleotide", id=",".join(ids), rettype="fasta", retmode="text")
    seq_records = list(SeqIO.parse(StringIO(fetch_handle.read()), "fasta"))
    print(f"[INFO] Successfully retrieved {len(seq_records)} sequences.")
    return [str(rec.seq) for rec in seq_records]

def find_similarity(person1, people, k=10, cutoff=0.5):
    """
    Compute similarity between one sequence and a list of others using k-mer Jaccard.
    Uses multiset counts (bag-of-kmers) for more robustness.
    """
    from collections import Counter

    def get_kmers(seq, k):
        return Counter([seq[i:i+k] for i in range(len(seq)-k+1)])

    def jaccard_multiset(c1, c2):
        # intersection over union using min/max counts
        intersection = sum((c1 & c2).values())
        union = sum((c1 | c2).values())
        return 1 - intersection / union if union else 1.0

    print(f"[INFO] Generating {k}-mers for Person 1...")
    set1 = get_kmers(person1, k)

    print(f"[INFO] Comparing Person 1 against {len(people)} other sequences...")
    similarities = []
    for i, person in enumerate(people):
        print(f"   -> Processing Person {i+2}...")
        set2 = get_kmers(person, k)
        dist = jaccard_multiset(set1, set2)
        similarities.append((i, dist))
    
    similarities.sort(key=lambda x: x[1])
    
    print("[INFO] Similarity results (sorted by distance):")
    for idx, dist in similarities:
        print(f"Person {idx+1}: Jaccard distance = {dist:.3f}")
        if dist < cutoff:
            print("  -> Might be related")

# demo
print("Starting run...")
print("This will take a while, especially because it's in Python")
samples = fetch_sequences("BRCA1[Gene] AND Homo sapiens[Organism]", max_samples=10)
if samples:
    person1 = samples[0]
    people = samples[1:]
    print("[INFO] Beginning similarity analysis...")
    find_similarity(person1, people)
print("Done running.")


[INFO] Starting demo run...
[INFO] Searching NCBI for query: 'BRCA1[Gene] AND Homo sapiens[Organism]' (max 10)...
[INFO] Found 10 sequence IDs.
[INFO] Fetching 10 sequences from NCBI in bulk...
[INFO] Successfully retrieved 10 sequences.
[INFO] Beginning similarity analysis...
[INFO] Generating 10-mers for Person 1...
[INFO] Comparing Person 1 against 9 other sequences...
   -> Processing Person 2...
   -> Processing Person 3...
   -> Processing Person 4...
   -> Processing Person 5...
   -> Processing Person 6...
   -> Processing Person 7...
   -> Processing Person 8...
   -> Processing Person 9...
   -> Processing Person 10...
[INFO] Similarity results (sorted by distance):
Person 1: Jaccard distance = 0.035
  -> Might be related
Person 2: Jaccard distance = 0.998
Person 9: Jaccard distance = 1.000
Person 5: Jaccard distance = 1.000
Person 8: Jaccard distance = 1.000
Person 4: Jaccard distance = 1.000
Person 3: Jaccard distance = 1.000
Person 6: Jaccard distance = 1.000
Person 7: Jac